# Stripe Explanation

This guide will walk through a minimal example of working with a Stripe one-time payment link and webhook.

To get started we can import the stripe library and authenticate with a stripe API key that you can get from the stripe web UI.

In [ ]:
#| export
from fasthtml.common import *
import os

## Stripe Authentication

In [ ]:
#| export
import stripe

In [ ]:
#| export
stripe.api_key = os.environ.get("STRIPE_SECRET_KEY")
DOMAIN_URL = os.environ.get("DOMAIN_URL", "http://localhost:5001")

You can get this api key from the Stripe Dashboard by going to [this url](https://dashboard.stripe.com/test/apikeys).

> Note: Make sure you have test mode turned on

![](StripeDashboard_API_Key.png)

Make sure you are using a test key for this tutorial

In [ ]:
assert 'test' in stripe.api_key

## Pre-app setup

### Create a product

These are steps that you can do to programatically create a product an associate a price to that product.  Typically this is not something you do dynamically in the app, but rather something you set up one time.

In [ ]:
#| export
product = stripe.Product.create(name="Demo Product")
price = stripe.Price.create(product=product.id, unit_amount=1999, currency="usd")

In [ ]:
print(f"Price ID = {price.id}")

Price ID = price_1Qz3hRKGhqIw9PXmD2B1PTHS


We will need the price ID later - we've printed it for you here but you can find it on your dashboard as well.

### Create a webook

A webhook is a way for Stripe (the payment processor) to notify your application when something happens with a payment. Think of it like a delivery notification: when a customer completes a payment, Stripe needs to tell your application so you can update your records, send confirmation emails, or provide access to purchased content.

Since your application runs locally during development (only accessible on your computer), Stripe can't reach it directly. The Stripe CLI tool creates a secure tunnel that forwards these payment notifications from Stripe's servers to your local application. The webhook secret is like a password that ensures these notifications are actually coming from Stripe and not someone else pretending to be Stripe.


```bash
stripe listen --forward-to http://localhost:5001/webhook
```

Take the secret it gives you and set it to an environment variable.

```bash
export STRIPE_LOCAL_TEST_WEBHOOK_SECRET=<your-secret>
```

For a deployed app, you need a permenent webhook connection instead of the stripe CLI utility.

This code creates a permanent webhook connection between Stripe and your application. Unlike the local testing tool which creates a temporary tunnel, this establishes an official notification channel where Stripe will send real-time updates about payments to your application's /webhook URL.

The webhook is configured to notify your app about specific payment events (completed checkouts, successful payments, and failed payments). Stripe authenticates these notifications using a secret key, ensuring they're legitimate. This is essential for production environments where your app needs to automatically respond to payment activities without manual intervention.

```python
stripe.WebhookEndpoint.create(
        url=f"{DOMAIN_URL}/webhook",
        enabled_events=["checkout.session.completed",
                        "payment_intent.succeeded",
                        "payment_intent.payment_failed"],
        description="Webhook for payment notifications")
WEBHOOK_SECRET = webhook_endpoint.secret
print(f"Created webhook endpoint: {webhook_endpoint.id}")
```

## App

### Setup to have the right information

In order to accept a payment, you need to know who is making the payment.

There are many ways to accomplish this, for example using [oauth]() or a form.  For this example we will start with hardcoding an email address into  a session to simulate what it would look like with oauth.

In [ ]:
#| export
def before(sess): sess['auth'] = 'hamel@hamel.com'
bware = Beforeware(before, skip=['/webhook'])
app, rt = fast_app(before=bware)

We will need our webhook secret that was created.  For this tutorial, we will be using the local development environment variable that was created above.

In [ ]:
#| export
WEBHOOK_SECRET = os.getenv("STRIPE_LOCAL_TEST_WEBHOOK_SECRET")

### Payment Setup

We need 2 things first:
1. A button for users to click to pay
1. A route that gives stripe the information it needs to process the payment

In [ ]:
#| export
@rt("/")
def home(sess):
    auth = sess['auth']
    return Titled(
        "Buy Now", 
        Div(H2("Demo Product - $19.99"),
            P(f"Welcome, {auth}"),
            Button("Buy Now", hx_post="/create-checkout-session", hx_swap="none")))

In [ ]:
#| export
@rt("/create-checkout-session", methods=["POST"])
async def create_checkout_session(sess):
    checkout_session = stripe.checkout.Session.create(
        line_items=[{'price': price.id, 'quantity': 1}],
        mode='payment',
        payment_method_types=['card'],  # Only allow card payments
        customer_email=sess['auth'],
        metadata={'app_name': 'MyFastApp', 
                  'AnyOther': 'Metadata',},
        # CHECKOUT_SESSION_ID is a special variable Stripe fills in for you
        success_url=DOMAIN_URL + '/success?checkout_sid={CHECKOUT_SESSION_ID}',
        cancel_url=DOMAIN_URL + '/cancel')
    return Redirect(checkout_session.url)

This section creates two key components: a simple webpage with a "Buy Now" button, and a function that handles what happens when that button is clicked. 

When a customer clicks "Buy Now," the app creates a Stripe checkout session (essentially a payment page) with product details, price, and customer information. Stripe then takes over the payment process, showing the customer a secure payment form. After payment is completed or canceled, Stripe redirects the customer back to your app using the success or cancel URLs you specified. This approach keeps sensitive payment details off your server, as Stripe handles the actual transaction.

### Post-Payment Processing

After a payment is processed, we need to handle both successful payments and cancellations, as well as listen for webhook notifications from Stripe.

The success and cancel routes provide immediate feedback to customers after they complete or abandon a payment. When a customer successfully pays, they're redirected to a confirmation page that displays their session ID. If they cancel, they see a simple cancellation message.

In [ ]:
#| export
@rt("/success")
def success(sess, checkout_sid:str):    
    try:
        # Verify the session exists and is paid
        email = sess['auth']
        checkout_session = stripe.checkout.Session.retrieve(checkout_sid) 
        if checkout_session.payment_status != 'paid': return Redirect("/")
        if checkout_session.customer_email != email: return Redirect("/")
        
        # Get payment record from database
        payment = payments.fetchone("checkout_session_id=?", (checkout_sid,))
        if not payment: return Titled("Error", P("Payment not found"))
        
        return Titled(
            "Success",
            Div(H2("Payment Successful!"),
                P(f"Thank you for your purchase, {email}"),
                P(f"Amount Paid: ${payment.amount / 100:.2f}"),
                P(f"Status: {payment.payment_status}"),
                P(f"Transaction ID: {payment.checkout_session_id}"),
                A("Back to Home", href="/")))

    except stripe.error.StripeError: return Redirect("/")
    
@rt("/cancel")
def cancel():
    return Titled(
        "Cancelled",
        Div(H2("Payment Cancelled"),
            P("Your payment was cancelled."),
            A("Back to Home", href="/")))

The webhook route is where Stripe sends automated notifications about payment events. When a payment is completed, Stripe sends a secure notification to this endpoint. The code verifies this notification is legitimate using the webhook secret, then processes the event data - extracting information like the customer's email and payment status. This allows your application to automatically update user accounts, trigger fulfillment processes, or record transaction details without manual intervention.

In [ ]:
#| export
# Database Table
class Payment:
    checkout_session_id: str  # Stripe checkout session ID (primary key)
    email: str 
    amount: int  # Amount paid in cents
    payment_status: str  # paid, pending, failed
    created_at: int # Unix timestamp
    metadata: str  # Additional payment metadata as JSON

In [ ]:
#| export
db = Database("stripe_payments.db")
payments = db.create(Payment, pk='checkout_session_id', transform=True)

In [ ]:
#| export
@rt("/webhook")
async def post(req):
    payload = await req.body()
    # Verify the event came from Stripe
    try:
        event = stripe.Webhook.construct_event(
            payload, req.headers.get("stripe-signature"), WEBHOOK_SECRET)
    except Exception as e:
        print(f"Webhook error: {e}")

    if event and event.type == "checkout.session.completed":
        event_data = event.data.object
        if event_data.metadata.get('app_name') == 'MyFastApp':
            payments.insert(Payment(
                checkout_session_id=event_data.id,
                email=event_data.customer_email,
                amount=event_data.amount_total,
                payment_status=event_data.payment_status,
                created_at=event_data.created,
                metadata=str(event_data.metadata)))
            print(f"Payment recorded for user: {event_data.customer_email}")

This image shows Stripe's payment page that customers see after clicking the "Buy Now" button. When your app redirects to the Stripe checkout URL, Stripe displays this secure payment form where customers enter their card details. For testing purposes, you can use Stripe's test card number (4242 4242 4242 4242) with any future expiration date and any 3-digit CVC code. This test card will successfully process payments in test mode without charging real money. The form shows the product name and price that were configured in your Stripe session, providing a seamless transition from your app to the payment processor and back again after completion.

![](StripePaymentPage.jpg)

Once you have processed the payemnts you can see each record in the sqlite database that was stored in the webhook.

In [ ]:
payments()

[Payment(checkout_session_id='cs_test_a1ExKxmnvZchjoPWAeQ2oPh1v370JxtEkQcSKFaK862WmlZhU7jfKdYdBe', email='hamel@hamel.com', amount=1999, payment_status='paid', created_at=1741126319, metadata='{\n  "AnyOther": "Metadata",\n  "app_name": "MyFastApp"\n}'),
 Payment(checkout_session_id='cs_test_a1ySporq7nz1im1nmU05fNN9viBXina7gk2yNEChmdXkLpkoDWHrA6Vmmi', email='hamel@hamel.com', amount=1999, payment_status='paid', created_at=1741126448, metadata='{\n  "AnyOther": "Metadata",\n  "app_name": "MyFastApp"\n}')]

In [ ]:
#| export
serve()